In [ ]:
# %%
# 📘 5. Parse Labelized CSVs (Already in short class format, like "Qc", "9s", "2h")

def parse_labelized_csv(csv_path: str, base_dir: str) -> pd.DataFrame:
    """
    Parse the Labelized CSV dataset where class names are already short (e.g. 'Qc', '9s').
    Converts VOC-style bounding boxes to YOLO format and returns a DataFrame.
    """
    if not os.path.exists(csv_path):
        print(f"⚠️ Missing file: {csv_path}")
        return pd.DataFrame()

    df_csv = pd.read_csv(csv_path)
    all_rows = []
    skipped = 0

    for _, row in df_csv.iterrows():
        fname = str(row["filename"]).strip()
        label = str(row["class"]).strip()

        if label not in yolo_class_to_id:
            print(f"⚠️ Skipping unknown class '{label}' in {fname}")
            skipped += 1
            continue

        try:
            img_w = float(row["width"])
            img_h = float(row["height"])
            if img_w == 0 or img_h == 0:
                print(f"⚠️ Skipping {fname}: invalid size ({img_w}x{img_h})")
                skipped += 1
                continue
        except Exception as e:
            print(f"⚠️ Skipping {fname}: error reading width/height ({e})")
            skipped += 1
            continue

        x_center, y_center, width, height = convert_voc_to_yolo(
            float(row["xmin"]), float(row["ymin"]),
            float(row["xmax"]), float(row["ymax"]),
            img_w, img_h
        )

        class_id = yolo_class_to_id[label]
        image_path = str(Path(base_dir) / fname)

        all_rows.append([image_path, class_id, label, x_center, y_center, width, height, img_w, img_h])

    df = pd.DataFrame(
        all_rows,
        columns=["image", "class_id", "class_name", "bbox_x_center", "bbox_y_center", "bbox_width", "bbox_height", "image_width", "image_height"]
    )

    print(f"{csv_path}: parsed {len(df)} valid annotations, skipped {skipped}.")
    return df


# Paths
labelized_train_csv = "datasets/Playing-Cards-Labelized-Dataset/train_cards_label.csv"
labelized_test_csv  = "datasets/Playing-Cards-Labelized-Dataset/test_cards_label.csv"

labelized_train_base = "datasets/Playing-Cards-Labelized-Dataset/train"
labelized_test_base  = "datasets/Playing-Cards-Labelized-Dataset/test"

# Load
labelized_train_df = parse_labelized_csv(labelized_train_csv, labelized_train_base)
labelized_test_df  = parse_labelized_csv(labelized_test_csv, labelized_test_base)

labelized_df = pd.concat([labelized_train_df, labelized_test_df], ignore_index=True)
print(f"Labelized total: {len(labelized_df)} annotations")

# Save
labelized_train_df.to_csv("datasets/Playing-Cards-Labelized-Dataset/labelized_train_converted.csv", index=False)
labelized_test_df.to_csv("datasets/Playing-Cards-Labelized-Dataset/labelized_test_converted.csv", index=False)
labelized_df.to_csv("datasets/Playing-Cards-Labelized-Dataset/labelized_all_converted.csv", index=False)


datasets/Playing-Cards-Labelized-Dataset/train_cards_label.csv: parsed 40089 valid annotations, skipped 0.
datasets/Playing-Cards-Labelized-Dataset/test_cards_label.csv: parsed 8004 valid annotations, skipped 0.
Labelized total: 48093 annotations


In [ ]:
# %%
# 📘 6. Parse "The-Complete-Playing-Card-Dataset" (YOLO format, with only .txt and images)

import re
from PIL import Image

COMPLETE_CLASSES = [
    '10c', '10d', '10h', '10s',
    '2c', '2d', '2h', '2s',
    '3c', '3d', '3h', '3s',
    '4c', '4d', '4h', '4s',
    '5c', '5d', '5h', '5s',
    '6c', '6d', '6h', '6s',
    '7c', '7d', '7h', '7s',
    '8c', '8d', '8h', '8s',
    '9c', '9d', '9h', '9s',
    'Ac', 'Ad', 'Ah', 'As',
    'Jc', 'Jd', 'Jh', 'Js',
    'Kc', 'Kd', 'Kh', 'Ks',
    'Qc', 'Qd', 'Qh', 'Qs'
]
COMPLETE_CLASS_TO_ID = {name.lower(): idx for idx, name in enumerate(COMPLETE_CLASSES)}

def parse_complete_yolo_dataset(base_dir: str) -> pd.DataFrame:
    base_dir = Path(base_dir)
    labels_dir = base_dir / "YOLO_Annotations" / "YOLO_Annotations"
    images_dir = base_dir / "Images" / "Images"

    if not labels_dir.exists():
        print(f"⚠️ Labels folder not found: {labels_dir}")
        return pd.DataFrame()

    all_rows = []
    txt_files = sorted(labels_dir.glob("*.txt"))

    for txt_file in txt_files:
        stem = txt_file.stem  # e.g. "10C15", "AC12"
        match = re.match(r"([0-9]{1,2}[cdhsCDHS]|[AJQKajqk][cdhsCDHS])", stem)
        if not match:
            print(f"⚠️ Could not infer class from {txt_file.name}")
            continue

        class_name = match.group(1).lower()
        if class_name not in COMPLETE_CLASS_TO_ID:
            print(f"⚠️ Unknown class '{class_name}' in {txt_file.name}")
            continue

        class_id = COMPLETE_CLASS_TO_ID[class_name]
        image_path = find_image(images_dir, stem)
        if not image_path:
            print(f"⚠️ Missing image for {stem}")
            continue

        with Image.open(image_path) as im:
            img_w, img_h = im.width, im.height

        with open(txt_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                _, x_center, y_center, width, height = parts
                all_rows.append([
                    str(image_path), class_id, class_name,
                    float(x_center), float(y_center), float(width), float(height),
                    float(img_w), float(img_h)
                ])

    df = pd.DataFrame(
        all_rows,
        columns=[
            "image", "class_id", "class_name",
            "bbox_x_center", "bbox_y_center",
            "bbox_width", "bbox_height",
            "image_width", "image_height"
        ]
    )

    print(f"✅ Parsed {len(df)} annotations from {labels_dir}")
    return df


# Esegui la conversione
complete_base = "datasets/The-Complete-Playing-Card-Dataset"
complete_df = parse_complete_yolo_dataset(complete_base)

# Salva il CSV risultante
complete_df.to_csv("datasets/The-Complete-Playing-Card-Dataset/complete_converted.csv", index=False)

# Mostra un’anteprima
display(complete_df.head())


⚠️ Could not infer class from JOKER0.txt
⚠️ Could not infer class from JOKER1.txt
⚠️ Could not infer class from JOKER10.txt
⚠️ Could not infer class from JOKER11.txt
⚠️ Could not infer class from JOKER12.txt
⚠️ Could not infer class from JOKER13.txt
⚠️ Could not infer class from JOKER14.txt
⚠️ Could not infer class from JOKER15.txt
⚠️ Could not infer class from JOKER16.txt
⚠️ Could not infer class from JOKER17.txt
⚠️ Could not infer class from JOKER18.txt
⚠️ Could not infer class from JOKER19.txt
⚠️ Could not infer class from JOKER2.txt
⚠️ Could not infer class from JOKER20.txt
⚠️ Could not infer class from JOKER21.txt
⚠️ Could not infer class from JOKER22.txt
⚠️ Could not infer class from JOKER23.txt
⚠️ Could not infer class from JOKER24.txt
⚠️ Could not infer class from JOKER25.txt
⚠️ Could not infer class from JOKER26.txt
⚠️ Could not infer class from JOKER27.txt
⚠️ Could not infer class from JOKER28.txt
⚠️ Could not infer class from JOKER29.txt
⚠️ Could not infer class from JOKER3.

,image,class_id,class_name,bbox_x_center,bbox_y_center,bbox_width,bbox_height,image_width,image_height
0,datasets/The-Complete-Playing-Card-Dataset/Ima...,0,10c,0.325087,0.665365,0.083767,0.039641,4608.0,3456.0
1,datasets/The-Complete-Playing-Card-Dataset/Ima...,0,10c,0.326280,0.623698,0.073134,0.069734,4608.0,3456.0
2,datasets/The-Complete-Playing-Card-Dataset/Ima...,0,10c,0.559462,0.498264,0.092448,0.063079,4608.0,3456.0
3,datasets/The-Complete-Playing-Card-Dataset/Ima...,0,10c,0.513997,0.449508,0.051432,0.068576,4608.0,3456.0
4,datasets/The-Complete-Playing-Card-Dataset/Ima...,0,10c,0.307834,0.513889,0.055339,0.079282,4608.0,3456.0
